# Poly2Graph Tutorial

In [ ]:
import numpy as np
import networkx as nx
import sympy as sp
from sympy.polys.polytools import Poly

import matplotlib.pyplot as plt

try:
    import poly2graph as p2g
except ImportError:
    !pip install poly2graph
    import poly2graph as p2g
print('p2g version:', p2g.__version__)
print('available functions:', p2g.__all__)

## Spectral Graph Construction (`p2g.SpectralGraph`)

In [ ]:
# always start by initializing the symbols for k, z, and E
k = sp.symbols('k', real=True)
z, E = sp.symbols('z E', complex=True)

### A generic **one-band** example:

characteristic polynomial:
$$P(E,z) := h(z) - E = z^4 -z -z^{-2} -E$$
Its Bloch Hamiltonian (Fourier transformed Hamiltonian in momentum space) is a scalar function:
$$h(z) = z^4 - z - z^{-2}$$
where the phase factor is defined as $z:=e^{ik}$.
Expressed in terms of crystal momentum $k$:
$$h(k) = e^{4ik} - e^{ik} - e^{-2ik}$$

The valid input formats to initialize a `p2g.SpectralGraph` object are:
1. Characteristic polynomial in terms of `z` and `E`:
   - as a string of the Poly in terms of `z` and `E`
   - as a `sympy`'s `Poly` (`sympy.polys.polytools.Poly`) with {`z`, `1/z`, `E`} as generators
2. Bloch Hamiltonian in terms of `k` or `z`
   - as a `sympy` `Matrix` in terms of `k`
   - as a `sympy` `Matrix` in terms of `z`

All the following `characteristic`s are valid and will initialize to the same characteristic polynomial and therefore produce the same spectral graph

In [ ]:
char_poly_str = '-z**-2 - E - z + z**4'

char_poly_Poly = Poly(
    -z**-2 - E - z + z**4,
    z, 1/z, E # generators are z, 1/z, E
)

phase_k = sp.exp(sp.I*k)
char_hamil_k = sp.Matrix([-phase_k**2 - phase_k + phase_k**4])

char_hamil_z = sp.Matrix([-z**-2 - z + z**4])

Let us just use the string to initialize and see a set of properties that are computed automatically

In [ ]:
sg = p2g.SpectralGraph(char_poly_str, k=k, z=z, E=E)

Characteristic polynomial:

In [ ]:
sg.ChP

Bloch Hamiltonian:
- For one-band model, it is a unique, rank-0 matrix (scalar)

In [ ]:
sg.h_k

In [ ]:
sg.h_z

The Frobenius companion matrix of `P(E)(z)`:
- treating `E` as parameter and `z` as variable
- Its eigenvalues are the roots of the characteristic polynomial at a fixed complex energy `E`. Thus it is useful to calculate the GBZ (generalized Brillouin zone), the spectral potential (Ronkin function), etc.

In [ ]:
sg.companion_E

In [ ]:
print('Number of bands:', sg.num_bands)
print('Max hopping length to the right:', sg.poly_p)
print('Max hopping length to the left:', sg.poly_q)

A real-space Hamiltonian of a finite chain and its energy spectrum:

In [ ]:
H = sg.real_space_H(
    N=40,        # number of unit cells
    pbc=False,   # open boundary conditions
    max_dim=500  # maximum dimension of the Hamiltonian matrix (for numerical accuracy)
)

energy = np.linalg.eigvals(H)

fig, ax = plt.subplots(figsize=(3, 3))
ax.plot(energy.real, energy.imag, 'k.', markersize=5)
ax.set(xlabel='Re(E)', ylabel='Im(E)', \
xlim=sg.spectral_square[:2], ylim=sg.spectral_square[2:])
plt.tight_layout()
# plt.savefig('./assets/finite_spectrum_one_band.png', dpi=300)
plt.show()

**The set of spectral functions** (whose values plotted on the complex energy square, returned as a 2D array):

- **Density of States (DOS)**

  Defined as the number of states per unit energy area in the complex energy plane.
  $$\rho(E) = \lim_{N\to\infty}\sum_n \frac{1}{N} \delta(E-\epsilon_n)$$
  where $\epsilon_n$ are the eigenvalues of the Hamiltonian $H$.
  Imagine to assign electric charge $1/N$ to each eigenvalue $\epsilon_n$, then the density of states $\rho(E)$ is treated as a *charge density*, therefore can be interpreted as the laplacian of a *spectral potential* $\Phi(E)$:
  $$\rho(E) = -\frac{1}{2\pi} \Delta \Phi(E)$$
  $\Delta = \partial_{\text{Re} E}^2 + \partial_{\text{Im} E}^2$ is the Laplacian operator on the complex energy plane. Laplacian operator extracts curvature; thus, geometrically speaking, the loci of spectral graph $\mathcal{G}$ resides on the *ridges* of the Coulomb potential landscape.

- **Spectral Potential (Ronkin function)**

  It can be proven that the spectral potential $\Phi(E)$ can be efficiently computed from the roots $|z_i(E)|$ of the characteristic polynomial $P(E)(z)$ and the leading coefficient $a_q(E)$ at a complex energy $E$:
  $$ \begin{aligned}
  \Phi(E) &= - \lim_{N\to\infty} \sum_{\epsilon_n} \log|E-\epsilon_n| \\
  &= - \int \rho(E')\log|E-E'| \; d^2E' \\
  &= - \log|a_q(E)| - \sum_{i=p+1}^{p+q} \log|z_i(E)| \\
  \end{aligned} $$

- Graph Skeleton (Binarized DOS)

In [ ]:
phi, dos, binaried_dos = sg.spectral_images(device='/gpu:0') # default is '/cpu:0'
# the computation bottleneck is implemented in tensorflow

fig, axes = plt.subplots(1, 3, figsize=(8, 3), sharex=True, sharey=True)
axes[0].imshow(phi, extent=sg.spectral_square, cmap='Spectral_r')
axes[0].set(xlabel='Re(E)', ylabel='Im(E)', title='Spectral Potential')
axes[1].imshow(dos, extent=sg.spectral_square, cmap='viridis')
axes[1].set(xlabel='Re(E)', title='Density of States')
axes[2].imshow(binaried_dos, extent=sg.spectral_square, cmap='gray')
axes[2].set(xlabel='Re(E)', title='Graph Skeleton')
plt.tight_layout()
# # plt.savefig('./assets/spectral_images_one_band.png', dpi=300)
plt.show()

In [ ]:
### interactive visualization of spectral potential
import plotly.graph_objects as go
E_re = np.linspace(*sg.spectral_square[:2], phi.shape[0])
E_im = np.linspace(*sg.spectral_square[2:], phi.shape[1])
fig = go.Figure(data=[go.Surface(z=phi, x=E_re, y=E_im, 
                opacity=0.6, colorscale='Spectral_r')])
fig.update_layout(
    scene=dict(
        aspectratio=dict(x=1.5, y=1, z=.8),
        xaxis_title='Re(E)',yaxis_title='Im(E)',zaxis_title='Phi(E)',
    ),
    title='3D Surface Plot of Spectral Potential',
    width=700, height=600
)
fig.show()

#### The spectral graph $\mathcal{G}$

In [ ]:
graph = sg.spectral_graph(
    device='/gpu:0', # default is '/cpu:0'
    short_edge_threshold=20, 
    # ^ node pairs or edges with distance < threshold pixels are merged
)

fig, ax = plt.subplots(figsize=(3, 3))
pos = nx.get_node_attributes(graph, 'pos')
nx.draw_networkx_nodes(graph, pos, alpha=0.8, ax=ax,
            node_size=50, node_color='#A60628')
nx.draw_networkx_edges(graph, pos, alpha=0.8, ax=ax,
            width=5, edge_color='#348ABD')
plt.tight_layout()
# plt.savefig('./assets/spectral_graph_one_band.png', dpi=300)
plt.show()

### A generic **multi-band** example:

characteristic polynomial (four bands):
$$P(E,z) := \det(\textbf{h}(z) - E\;\textbf{I}) = z^2 + 1/z^2 + E z - E^4$$
One of its possible Bloch Hamiltonians in terms of $z$:
$$\textbf{h}(z)=\begin{pmatrix}
0 & 0 & 0 & z^2 + 1/z^2 \\
1 & 0 & 0 & z \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
\end{pmatrix}$$

In [ ]:
sg_multi = p2g.SpectralGraph("z**2 + 1/z**2 + E*z - E**4", k, z, E)

In [ ]:
sg_multi.ChP

Bloch Hamiltonian:
- For multi-band model, if the `p2g.SpectralGraph` is not initialized with a `sympy` `Matrix`, then `poly2graph` will use the companion matrix of the characteristic polynomial `P(z)(E)` (treating `z` as parameter and `E` as variable) as the Bloch Hamiltonian -- this is one of the set of possible band Hamiltonians that possesses the same energy spectrum and thus the same spectral graph.

In [ ]:
sg_multi.h_k

In [ ]:
sg_multi.h_z

In [ ]:
sg_multi.companion_E

In [ ]:
print('Number of bands:', sg_multi.num_bands)
print('Max hopping length to the right:', sg_multi.poly_p)
print('Max hopping length to the left:', sg_multi.poly_q)

In [ ]:
H_multi = sg_multi.real_space_H(
    N=40,        # number of unit cells
    pbc=False,   # open boundary conditions
    max_dim=500  # maximum dimension of the Hamiltonian matrix (for numerical accuracy)
)

energy_multi = np.linalg.eigvals(H_multi)

fig, ax = plt.subplots(figsize=(3, 3))
ax.plot(energy_multi.real, energy_multi.imag, 'k.', markersize=5)
ax.set(xlabel='Re(E)', ylabel='Im(E)', \
xlim=sg_multi.spectral_square[:2], ylim=sg_multi.spectral_square[2:])
plt.tight_layout()
# plt.savefig('./assets/finite_spectrum_multi_band.png', dpi=300)
plt.show()

In [ ]:
phi_multi, dos_multi, binaried_dos_multi = sg_multi.spectral_images(device='/gpu:0') # default is '/cpu:0'
# the computation bottleneck is implemented in tensorflow

fig, axes = plt.subplots(1, 3, figsize=(8, 3), sharex=True, sharey=True)
axes[0].imshow(phi_multi, extent=sg_multi.spectral_square, cmap='Spectral_r')
axes[0].set(xlabel='Re(E)', ylabel='Im(E)', title='Spectral Potential')
axes[1].imshow(dos_multi, extent=sg_multi.spectral_square, cmap='viridis')
axes[1].set(xlabel='Re(E)', title='Density of States')
axes[2].imshow(binaried_dos_multi, extent=sg_multi.spectral_square, cmap='gray')
axes[2].set(xlabel='Re(E)', title='Graph Skeleton')
plt.tight_layout()
# plt.savefig('./assets/spectral_images_multi_band.png', dpi=300)
plt.show()

#### The spectral graph $\mathcal{G}$

In [ ]:
graph_multi = sg_multi.spectral_graph(
    device='/gpu:0', # default is '/cpu:0'
    short_edge_threshold=20, 
    # ^ node pairs or edges with distance < threshold pixels are merged
)

fig, ax = plt.subplots(figsize=(3, 3))
pos_multi = nx.get_node_attributes(graph_multi, 'pos')
nx.draw(graph_multi, pos_multi, ax=ax, 
        node_size=10, node_color='#A60628', 
        edge_color='#348ABD', width=2, alpha=0.8)
plt.tight_layout()
# plt.savefig('./assets/spectral_graph_multi_band.png', dpi=300)
plt.show()

### Node and Edge Attributes of the Spectral Graph Object

The spectral graph is a `networkx.MultiGraph` object.

- Node Attributes
  1. `pos` : (2,)-numpy array
     - the position of the node $(\text{Re}(E), \text{Im}(E))$
  2. `dos` : float
     - the density of states at the node
  3. `potential` : float
     - the spectral potential at the node
- Edge Attributes
  1. `weight` : float
     - the weight of the edge, which is the **length** of the edge in the complex energy plane
  2. `pts` : (w, 2)-numpy array
     - the positions of the points constituting the edge, where `w` is the number of points along the edge, i.e., the length of the edge, equals `weight`
  3. `avg_dos` : float
     - the average density of states along the edge
  4. `avg_potential` : float
     - the average spectral potential along the edge

In [ ]:
node_attr = dict(graph.nodes(data=True))
edge_attr = list(graph.edges(data=True))

np.set_printoptions(precision=5, suppress=True, threshold=20, edgeitems=3)
print('The attributes of the first node\n', node_attr[0], '\n')
print('The attributes of the first edge\n', edge_attr[0][-1], '\n')

## Characteristic Polynomial Class (`p2g.CharPolyClass`)

- A class of parametrized non-Hermitian lattices
- Generate spectral properties, including **spectral graph** in parallel
- Optimized for computational efficiency and numerical stability

Let us add two parameters `{a,b}` to the aforementioned multi-band example and construct a `p2g.CharPolyClass` object.

In [ ]:
a, b = sp.symbols('a b', real=True)

cp = p2g.CharPolyClass(
    "z**2 + a/z**2 + b*E*z - E**4", 
    k=k, z=z, E=E,
    params={a, b}, # pass parameters as a set
)

View a few auto-computed properties:

In [ ]:
cp.ChP

In [ ]:
cp.h_k

In [ ]:
cp.h_z

In [ ]:
cp.companion_E

To get an array of spectral images or spectral graphs, we first prepare the values of the parameters `{a,b}`

In [ ]:
a_array = np.linspace(-2, 1, 6)
b_array = np.linspace(-1, 1, 6)
a_grid, b_grid = np.meshgrid(a_array, b_array)
param_dict = {a: a_grid, b: b_grid}
print('a_grid shape:', a_grid.shape,
    '\nb_grid shape:', b_grid.shape)

Note that **the value array of the parameters should have the same shape**, which is also **the shape of the output array of spectral images**

In [ ]:
phi_arr, dos_arr, binaried_dos_arr, spectral_square = \
    cp.spectral_images(param_dict=param_dict, device='/cpu:0')
print('phi_arr shape:', phi_arr.shape,
    '\ndos_arr shape:', dos_arr.shape,
    '\nbinaried_dos_arr shape:', binaried_dos_arr.shape)

In [ ]:
from mpl_toolkits.axes_grid1 import ImageGrid

fig = plt.figure(figsize=(13, 13))
grid = ImageGrid(fig, 111, nrows_ncols=(6, 6), axes_pad=0, 
                 label_mode='L', share_all=True)

for ax, (i, j) in zip(grid, [(i, j) for i in range(6) for j in range(6)]):
    ax.imshow(phi_arr[i, j], extent=spectral_square[i, j], cmap='Spectral_r')
    ax.set(xlabel='Re(E)', ylabel='Im(E)')
    ax.text(
        0.03, 0.97, f'a = {a_array[i]:.2f}, b = {b_array[j]:.2f}',
        ha='left', va='top', transform=ax.transAxes,
        fontsize=10, color='tab:red',
        bbox=dict(alpha=0.8, facecolor='white')
    )

plt.tight_layout()
# plt.savefig('./assets/ChP_spectral_potential_grid.png', dpi=72)
plt.show()

### An Array of Spectral Graphs

In [ ]:
graph_flat, param_dict_flat = cp.spectral_graph(
    param_dict=param_dict, device='/cpu:0',
    short_edge_threshold=20,
)
print(graph_flat[:5])
print(param_dict_flat)

Note that the spectral graph is a `networkx.MultiGraph` object, which cannot be directly returned as a multi-dimensional numpy array of `MultiGraph`, except for the case of 1D array.
Instead, we return a flattened list of `networkx.MultiGraph` objects, and the accompanying `param_dict_flat` is the dictionary that contains the corresponding flattened parameter values.

It's recommended to pass the values of the parameters as `vectors` (1D arrays) instead of higher dimensional `ND arrays` to avoid the overhead of reshaping the output and the difficulty to retrieve / postprocess the spectral graphs.